In [ ]:
!pip install ultralytics

In [ ]:
from ultralytics import YOLO
import os
import boto3
import shutil

In [ ]:
EPOCHS=int(os.environ.get("epochs", 50))
IMGSZ=640
NB_FROZEN_LAYER=0

DATA_CONFIG_PATH = 'utils/data.yaml'
LOCAL_DATASET_PATH = "basketball-dataset/"

BASE_MODEL_PATH = "models/yolo11n.pt"
TRAIN_MODEL_PATH = f"models/new-bb-yolo11n{NB_FROZEN_LAYER}.pt"
TRAIN_MODEL_PATH_ONNX = f"models/new-bb-yolo11n{NB_FROZEN_LAYER}.onnx"

AWS_ACCESS_KEY_ID = os.environ.get("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.environ.get("AWS_SECRET_ACCESS_KEY")
AWS_S3_BUCKET = os.environ.get("AWS_S3_BUCKET")
AWS_S3_ENDPOINT = os.environ.get("AWS_S3_ENDPOINT")

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [ ]:
session = boto3.Session(
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY
)
client = session.client("s3", endpoint_url=AWS_S3_ENDPOINT)

In [ ]:
client.download_file(AWS_S3_BUCKET, BASE_MODEL_PATH, "base-model.pt")

In [ ]:
if os.path.exists('runs'):
    shutil.rmtree('runs')
model = YOLO('base-model.pt')
results = model.train(data=DATA_CONFIG_PATH, epochs=EPOCHS, imgsz=IMGSZ, freeze=NB_FROZEN_LAYER, batch=-1)

In [ ]:
model = YOLO('runs/detect/train/weights/best.pt')

In [ ]:
model.export(format="onnx")

In [ ]:
model_path = "runs/detect/train/weights/best.onnx"

In [ ]:
client.upload_file(model_path, AWS_S3_BUCKET, TRAIN_MODEL_PATH_ONNX)

In [ ]:
client.upload_file('runs/detect/train/weights/best.pt', AWS_S3_BUCKET, TRAIN_MODEL_PATH)

In [ ]:
client.upload_file('runs/detect/train/results.csv', AWS_S3_BUCKET, 'results.csv')